# SignLanguageDetection_EmbeddedMachineLearning-RaspberryPi

Project to deploy a Machine Learning Model on a Rasberry Pi.  
Training a model to identify american sign language letters in images.

## First Time setup  
- You're expected to have [install](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tf-install)ed [tensorflow in your PC](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tf-install), check the References in the bottom for notes.
- After that, in your conda environment, there are a several dependencies you must install:
  - if you created your connda environment with `conda create -n signLanguageDetector pip python=3.9` for example.
  - run `conda activate signLanguageDetector`
  - `pip install wget hazm pandas` and other libraries that appear that might be needed.
- To download the dataset and the pre-trained-model run:

In [ ]:
import wget 
import zipfile
import os
import tarfile

print("Running first time setup...")

# check folders
ckckfldrs = os.listdir("./TensorFlow/workspace/training_demo/")

if all( i != "images" for i in ckckfldrs):
    os.mkdir("./TensorFlow/workspace/training_demo/images")
    # Downloading database
    print("thanks to https://public.roboflow.com/object-detection/american-sign-language-letters/1 for dataset")
    print("Downloading Dataset to ./TensorFlow/workspace/training_demo/images/...")
    url = 'https://public.roboflow.com/ds/wdx82NVcss?key=lyVASY8xq4'
    path = './TensorFlow/workspace/training_demo/images/'
    wget.download(url,out = path)

    print("\nUnzipping...")
    zipPath = os.listdir("./TensorFlow/workspace/training_demo/images/")
    zipPath = "./TensorFlow/workspace/training_demo/images/" + str(zipPath[0])
    print(zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("./TensorFlow/workspace/training_demo/images/")

    for item in os.listdir("./TensorFlow/workspace/training_demo/images/"): # loop through items in dir
        if item.endswith(".zip"): # check for ".zip" extension
            os.remove("./TensorFlow/workspace/training_demo/images/"+ item)
else:
    print("images folder already exists")

if all( i != "pre-trained-models" for i in ckckfldrs):
    os.mkdir("./TensorFlow/workspace/training_demo/pre-trained-models")
    # Downloading Pre-trained model
    print("using SSD ResNet50 V1 FPN 640x640 pre-trained model from https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md")
    print("Downloading pre-trained model to ./TensorFlow/workspace/training_demo/pre-trained-models/..")
    url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
    path = './TensorFlow/workspace/training_demo/pre-trained-models/'
    wget.download(url,out = path)

    print("\nUnzipping...")
    zipPath = os.listdir("./TensorFlow/workspace/training_demo/pre-trained-models/")
    zipPath = "./TensorFlow/workspace/training_demo/pre-trained-models/" + str(zipPath[0])
    tar = tarfile.open(zipPath, "r:gz")
    tar.extractall('./TensorFlow/workspace/training_demo/pre-trained-models/')
    tar.close()

    for item in os.listdir("./TensorFlow/workspace/training_demo/pre-trained-models/"): # loop through items in dir
        if item.endswith(".tar.gz"): # check for ".zip" extension
            os.remove("./TensorFlow/workspace/training_demo/pre-trained-models/"+ item)
else:
    print("pre-trained-models folder already exists")


## [Generate .record files from labeled dataset](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records)
(if you're using the same dataset as me, they're already generated!)  
Something like:

```bash
# Create train data:
python ./TensorFlow/scripts/preprocessing/generate_tfrecord.py -x [PATH_TO_IMAGES_FOLDER]/train -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map.pbtxt -o [PATH_TO_ANNOTATIONS_FOLDER]/train.record

# Create test data:
python ./TensorFlow/scripts/preprocessing/generate_tfrecord.py -x [PATH_TO_IMAGES_FOLDER]/test -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map.pbtxt -o [PATH_TO_ANNOTATIONS_FOLDER]/test.record

# For example
# python generate_tfrecord.py -x C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/images/train -l C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt -o C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/annotations/train.record
# python generate_tfrecord.py -x C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/images/test -l C:/Users/sglvladi/Documents/Tensorflow2/workspace/training_demo/annotations/label_map.pbtxt -o C:/Users/sglvladi/Documents/Tensorflow/workspace/training_demo/annotations/test.record
```

## Train the model

In [ ]:
import os
if all( i != "model_main_tf2.py" for i in os.listdir(".")):
    os.chdir('./TensorFlow/workspace/training_demo/') # Change the working directory
    print(f"working directory now: {os.getcwd()}")
%run -i 'model_main_tf2.py' --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config
os.chdir('../../../')
print(f"working directory now: {os.getcwd()}")

## [Monitor Training Job Progress using TensorBoard](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#tensorboard-sec)
1. While the model trains, you can 
   1. cd into the training_demo folder
   2. run `tensorboard --logdir=models/my_ssd_resnet50_v1_fpn` 
   3. access http://localhost:6006/ in your browser

## [Evaluating the Model](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#evaluating-the-model-optional)
1. After the model has been trained, you can see how good it does by:
   1. cd into the training_demo folder
   2. run `python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --checkpoint_dir=models/my_ssd_resnet50_v1_fpn`
   3. access http://localhost:6006/ in your browser
   4. seing in the images section, the eval:side_by_side images to see how good it is detecting

## [Exporting the Trained Model](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#exporting-a-trained-model) 

In [ ]:
import os
if all( i != "exporter_main_v2.py" for i in os.listdir(".")):
    os.chdir('./TensorFlow/workspace/training_demo/') # Change the working directory
    print(f"working directory now: {os.getcwd()}")
%run -i './exporter_main_v2.py' --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/my_model
os.chdir('../../../')
print(f"working directory now: {os.getcwd()}")

## [Detect Objects Using Your Webcam](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/object_detection_camera.html#detect-objects-using-your-webcam)

### Load the model

In [1]:
DATA_DIR = os.path.join(os.getcwd(), 'TensorFlow/workspace/training_demo/')
MODELS_DIR = os.path.join(DATA_DIR, 'exported-models')
MODEL_NAME = 'my_model_american_alphabet_sign_language'
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

### Load label map data (for plotting)

In [2]:
PATH_TO_LABELS = os.path.join(os.getcwd(), 'TensorFlow/workspace/training_demo/annotations/label_map.pbtxt')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

## Define the video stream

In [3]:
import cv2

cap = cv2.VideoCapture(0)

[ WARN:0@0.021] global /io/opencv/modules/videoio/src/cap_v4l.cpp (889) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


## Putting everything together

In [6]:
import numpy as np

while True:
    # Read frame from camera
    ret, image_np = cap.read()

    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    # Display output
    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

StagingError: in user code:

    File "/tmp/ipykernel_90630/1209127725.py", line 34, in detect_fn  *
        image, shapes = detection_model.preprocess(image)
    File "/home/yeshey/anaconda3/envs/signLanguageDetector/lib/python3.9/site-packages/object_detection/meta_architectures/ssd_meta_arch.py", line 482, in preprocess  *
        normalized_inputs = self._feature_extractor.preprocess(inputs)
    File "/home/yeshey/anaconda3/envs/signLanguageDetector/lib/python3.9/site-packages/object_detection/models/ssd_resnet_v1_fpn_keras_feature_extractor.py", line 203, in preprocess  *
        if resized_inputs.shape.as_list()[3] == 3:

    IndexError: list index out of range


# [Convert a model to Lite model](https://www.tensorflow.org/lite/convert)

In [ ]:
import tensorflow as tf

saved_model_dir = "./TensorFlow/workspace/training_demo/exported-models/my_model_american_alphabet_sign_language/saved_model"

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

## References

### Training the model
1. Following [*Training Custom Object Detector*](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) tutorial
     - [TensorFlow Installation](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tf-install) Notes:
       - With Anaconda installed, use `conda info --envs` to see created environments
       - Before working on the project, do `conda activate signLanguageDetector` to activate the environment every time
       - Install cuda and cudnn tu use GPU on manjaro with `sudo pacman -S cuda cudnn`
       - For Object detection API, added the [TensorFlow Models repository](https://github.com/tensorflow/models) as a git submodule with the command `git submodule add https://github.com/tensorflow/models ./TensorFlow/models/`
       - To test if everything worked out in the installation you can run `python TensorFlow/models/research/object_detection/builders/model_builder_tf2_test.py`
      - Dataset from roboflow.com: [American Sign Language Letters Dataset](https://public.roboflow.com/object-detection/american-sign-language-letters/1)
      - Had to do [this](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md#python-package-installation) before training to fix an error
2. [Detect Objects Using Your Webcam](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/object_detection_camera.html#detect-objects-using-your-webcam)